### DNNスクラッチコード

### データのロード

mnistの手書き文字認識問題

784ピクセルの値（0-255）から数字（0-9）を分類する。

In [1]:
import numpy as np
from keras.datasets import mnist
import gc
from sklearn.model_selection import train_test_split

# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# # プロトタイプなので100データだけ使用する
# x_train = x_train[:100]
# y_train = y_train[:100]
# y_label = y_train
# y_train = np.identity(10)[y_train]
# del x_test, y_test


# # 訓練とテストデータに分割
# x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=0)
# gc.collect()

/Users/kzfm/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


12

In [2]:
# カラー画像のデータセット
from keras.datasets import cifar10

(x_train_cifar10, y_train_cifar10), (x_test_cifar10, y_test_cifar10) = cifar10.load_data()

# プロトタイプなので100データだけ使用する
x_train_cifar10 = x_train_cifar10[:100]
y_train_cifar10 = y_train_cifar10[:100]
y_label_cifar10 = y_train_cifar10
y_train_cifar10 = np.identity(10)[y_train_cifar10]
del x_test_cifar10, y_test_cifar10

gc.collect()

print('x_train : {}'.format(x_train_cifar10.shape))
print('y_train : {}'.format(y_train_cifar10.shape))
print('y_label : {}'.format(y_label_cifar10.shape))

x_train : (100, 32, 32, 3)
y_train : (100, 1, 10)
y_label : (100, 1)


### 正答率を算出する関数

In [3]:
# # 正答率を算出する
# def accuracy_score(X, y, params):
#     y_pred =  predict(X, params)
#     y_pred_number = np.argmax(y_pred, axis=1)
    
#     y_label = np.argmax(y, axis=1)
    
#     data_size = X.shape[0]
    
#     correct_count = np.sum([y_label == y_pred_number]) 
#     score = correct_count / data_size * 100
    
#     return round(score, 2)

### レイヤークラス

親クラスとしてlayerクラスを作成。

以後、各機能のレイヤークラスはこの親クラスを継承することにする。


In [4]:
class Layer:
    def __init__(self, params={}):
        if 'input_shape' in params:
            self.in_shape = params['input_shape']
        else:
            self.in_shape = None
            
        if 'output_shape' in params:
            self.out_shape = params['output_shape']
        else:
            self.out_shape = None

### 最終層

分類問題用の最終層クラス（ソフトマックスとクロスエントロピーの結合層）

TODO: 正則化項がついていない。　すべての層の重みWを集計するメソッドをNetworkクラスに作成してそれを呼ばなくてはいけないため後回し。

回帰問題用に別クラスが必要になる。



### ソフトマックス関数

In [5]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)

### コスト関数

In [6]:
def cross_entropy_error(y, y_pred):
    
    data_size = y.shape[0]

    # クロスエントロピー誤差関数　y_predは０になりえるので -inf にならないためにすごく小さい補正値を入れる
    cross_entorpy = -np.sum(y * np.log(y_pred + 1e-7))
    
    error = cross_entorpy  / data_size
    return error

In [7]:
# 以前の正則化項つきのクロスエントロピー関数　いずれ使うのでそのままにしておく。
# def cost(y, y_pred, params, lam=0.01):
#     data_size = y.shape[0]
#     #  正則化項
#     weight_sum = sum([np.sum(matrix**2) for key, matrix in params.items() if "W" in key])
#     reg_term = (lam /2) * (weight_sum)
#     # クロスエントロピー誤差関数　y_predは０になりえるので -inf にならないためにすごく小さい補正値を入れる
#     cross_entorpy = -np.sum(y * np.log(y_pred + 1e-7))
    
#     cost = (cross_entorpy + reg_term) / data_size
#     return cost

In [8]:
class SoftmaxWithLoss(Layer):
    def __init__(self, params={}):
        super(SoftmaxWithLoss, self).__init__(params)
        self.loss = None # 損失関数
        self.y = None       # softmaxの出力
        self.t = None       # 教師データ（one-hot vector)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size # delta3に相当
        
        return dx

### アフィン変換層 (xW + b)

更新手法を
- sgd
- adagrad
- adam

と切り替えられる。

In [239]:
class Affine(Layer):
    
    def __init__(self, unit_size=100, params= {}):
        super(Affine, self).__init__(params)
        self.W = None
        self.b = None
        self.x = None
        self.unit_size = unit_size
        # パラメータの微分値
        self.dW = None
        self.db = None
        # 学習率のセット
        if 'lr' in params:
            self.lr = params['lr']
        else:
            self.lr = 0.01

        
    def initialize(self, in_shape, params={}):
        self.in_shape = in_shape # N F(特徴数)で来ると想定
        N, F = in_shape
        
        self.out_shape = (N, self.unit_size)
        
        # 重みの初期化
        # im2colを見越して２次元の重みとして実装
        self.W = np.random.randn(F, self.unit_size)
        self.b = np.zeros([1, self.unit_size])
        # TODO 初期化を選択できるように設定する　ひとまずはガウス初期化
#         if params['init'] == 'gauss':
        self.W *= 0.01
#         elif params['init'] == 'xavier':
#             # 入力層のユニット数 N * out_h * out_w ? 
#             self.W /= np.sqrt(self.out_shape[0] * self.out_shape[1] * self.out_shape[2])
#         else: # He
#             self.W = self.W / np.sqrt(self.out_shape[0] * self.out_shape[1] * self.out_shape[2]) * np.sqrt(2) 

        # オプティマイザの設定
        # optimizeメソッドをoptimizerによって切り替える。
        if 'optimizer' in params:
            if params['optimizer']=='sgd':
                self.optimize = self.update_sgd
            elif params['optimizer'] == 'adagrad':
                self.h = np.zeros_like(self.W)
                self.optimize = self.update_adagrad
            else: # params['optimizer'] == 'adam':
                self.m = np.zeros_like(self.W)
                self.v = np.zeros_like(self.W)
                self.beta1 = 0.9
                self.beta2 = 0.999
                self.optimize = self.update_adam
        else:
            params['optimizer'] = 'adam'
            self.optimize = self.update_adam
            
        # 次の層の初期化のために出力shapeを返す
        print("Affine : out {} optimizer {} unit {} ".format(self.out_shape, params['optimizer'], self.unit_size))
        return self.out_shape
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out
    
    def backward(self, dout=1):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx
    
    def update_sgd(self):
        self.W -= self.lr * self.dW
        self.b -= self.lr * self.db
    
    # adagrad 少しずつ更新量が減っていく
    def update_adagrad(self, lr = 0.01):
        self.h += self.dW ** 2
        self.W -= self.lr * self.dW / (np.sqrt(self.h) + 1e-7)
        self.b -= self.lr * self.db
        
    
    def update_adam(self, lr = 0.01):
        self.m = self.beta1 * self.m + (1- self.beta1) * self.dW
        self.v = self.beta2 * self.v + (1- self.beta2) * (self.dW * self.dW)
        
        m_hat = self.m / (1 - self.beta1)
        v_hat = self.v / (1 - self.beta2)
        
        self.W -= self.lr * m_hat / (np.sqrt(v_hat) + 1e-8)
        self.b -= self.lr * self.db
        

### 活性化関数層

活性化関数層は

- ReLU
- tanh
- シグモイド関数

を切り替えて使用できる。

In [207]:
class Activation(Layer):
    '''
    活性化関数を設定できる 
    'tanh'
    'sigmoid'
    'relu'
    '''
    def __init__(self, params):
        super(Activation, self).__init__(params)
        self.out = None
        self.mask = None
        # optimizeメソッドを
        if 'activation' in params:
            if params['activation']=='tanh':
                self.forward = self.forward_tanh
                self.backward = self.backward_tanh
            elif params['activation'] == 'sigmoid':
                self.forward = self.forward_sigmoid
                self.backward = self.backward_sigmoid
            else: # params['activation'] == 'relu':
                self.forward = self.forward_relu
                self.backward = self.backward_relu
        else:
            params['activation'] = 'relu'
            self.forward = self.forward_relu
            self.backward = self.backward_relu
            
    def initialize(self, in_shape, params={}):
        self.in_shape = in_shape # N H W Cで来ると想定
        self.out_shape = self.in_shape
            
        # 次の層の初期化のために出力shapeを返す
        print("Activation : out {}   func : {} ".format(self.out_shape, params['activation']))
        return self.out_shape
     
    def forward_relu(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out
    
    def backward_relu(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

    # tanh 
    def forward_tanh(self, x):
        out = np.tanh(x)
        self.out = out
        
        return out
    
    def backward_tanh(self, dout):
        dx = dout * (1 - np.tanh(dout)**2)
        
        return dx
    
    # sigmoid関数
    def forward_sigmoid(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward_sigmoid(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx
        

### バッチノーマリゼーション層

In [206]:
class BatchNorm(Layer):
    def __init__(self, params):
        super(BatchNorm, self).__init__(params)
        self.out = None
        self.beta = 0.0
        self.gamma = 1.0
        self.lr = params['lr']
        self.eps = 1e-8
    
        '''
        計算式は下記を参照
        https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html
        '''
    
    def initialize(self, in_shape, params={}):
        self.in_shape = in_shape # N H W Cで来ると想定
        self.out_shape = self.in_shape
            
        # 次の層の初期化のために出力shapeを返す
        print("BatchNorm : out {} lr {}  ".format(self.out_shape, self.lr))
        return self.out_shape
    
    def forward(self, x):
        #data_size, input_size = x.shape
        
        # 単に標準化する
        # out = (x - np.mean(x, axis=0)) / np.var()
        
        # step1: 平均を求める
        mu = np.mean(x, axis=0)
        
        # step2: 偏差
        self.xmu = x - mu
        
        # step3 : 偏差の２乗
        sq = self.xmu ** 2
        
        # step4 : 分散を求める
        self.var = np.var(x, axis=0)
        
        # step5 : 分散のルートを取った値を求める
        self.sqrtvar = np.sqrt(self.var + self.eps)
        
        # step6 : sqrtvarの逆数（invert）
        self.ivar = 1.0/ self.sqrtvar
        
        # step7 : 標準化した値
        self.xhat = self.xmu * self.ivar
        
        # step8
        gammax = self.gamma * self.xhat
        
        # step9
        out = gammax + self.beta

        return out

    
    def backward(self, dout=1):
        
        #get the dimensions of the input/output
        N, D = dout.shape
        
        # step9
        self.d_beta = np.sum(dout, axis=0)
        dgammax = dout #not necessary, but more understandable
        
        # step8
        self.d_gamma = np.sum(dgammax*self.xhat, axis=0)
        dxhat = dgammax * self.gamma
        
        # step7
        divar = np.sum(dxhat*self.xmu, axis=0)
        dxmu1 = dxhat * self.ivar
        
        # step6
        dsqrtvar = -1. /(self.sqrtvar**2) * divar
        
        # step5
        dvar = 0.5 * 1. / np.sqrt(self.var+self.eps) * dsqrtvar
        
        # step4
        dsq = 1. / N * np.ones((N, D)) * dvar
        
        # step3
        dxmu2 = 2 * self.xmu * dsq
        
        # step2
        dx1 = (dxmu1 + dxmu2)
        dmu = -1 * np.sum(dxmu1 + dxmu2, axis=0)
        
        # step1
        dx2 = 1. / N * np.ones((N, D)) * dmu
        
        # step0
        dx = dx1 + dx2
        
        return dx

    
    def optimize(self):
        self.gamma -= self.lr * self.d_gamma
        self.beta -= self.lr * self.d_beta
        

### ドロップアウト層

In [12]:
class Dropout(Layer):
    def __init__(self, params):
        super(Dropout, self).__init__(params)
        if 'dropout_ratio' in params:
            self.dropout_ratio = params['dropout_ratio']
        else:
            self.dropout_ratio = 0.5
        self.mask = None
    
    def forward(self, x, train_flg=True):
        if train_flg :
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio
            return x * self.mask
        else:
            return x * (1 - self.dropout_ratio)
    
    def backward(self, dout):
        return dout * self.mask

### 複数層を束ねるクラス(ネットワーククラス)

改良したいところ

addで層を追加（好きに追加できる。）

各層の親クラスを作成


In [13]:
# from collections import OrderedDict

# class Layers:
#     def __init__(self, params):
#         unit_size_list = [params['input_size']]
#         unit_size_list.extend(params['hidden_layer_list'])
#         unit_size_list.append(params['output_size'])
        
#         self.params = {}
        
#         # レイヤの生成
#         self.layers = OrderedDict()
        
#         # とりあえずここにべた書きで書けるようにする
        
# #         for i in range(1, len(unit_size_list)):
# #             # 重みの初期化
# #             init_W = np.random.randn(unit_size_list[i-1], unit_size_list[i])
# #             init_b = np.zeros([1, unit_size_list[i]])
# #             if params['init'] == 'gauss':
# #                 init_W *= 0.01
# #             elif params['init'] == 'xavier':
# #                 init_W /= np.sqrt(unit_size_list[i-1])
# #             else: # He
# #                 init_W = init_W / np.sqrt(unit_size_list[i-1]) * np.sqrt(2) 
                
# #             # アフィン変換層（Wx + b）を追加する
# #             self.layers['Affine' + str(i)] = Affine(init_W, init_b, params)
            
# #             # 最終層以外はバッチノーマリゼーション層と活性化関数層を追加する
# #             if i < (len(unit_size_list)-1):
# #                 if params['batch_norm'] == True:
# #                     self.layers['BatchNorm' + str(i)] = BatchNorm(params)
# #                 self.layers['Active' + str(i)] = Activation(params)
# # #                 if params['dropout_ratio'] > 0:
# # #                     self.layers['Dropout' + str(i)] = Dropout(params)
        
#         self.lastLayer = SoftmaxWithLoss()
        
#         # self.params['hidden_layer_num'] = len(unit_size_list)-1
        
#     def predict(self, x):
#         # forwardを繰り返す
#         # ソフトマックスを通さなくても答えは出るのでこれで予測とする 
#         # argmaxでラベルを取れる
#         for layer in self.layers.values():
#             x =layer.forward(x)

#         return x

#     def accuracy(self, x, t):
#         # 正答率を小数点第二桁で出力する
#         y_pred = self.predict(x)
#         y_pred = np.argmax(y_pred, axis=1)
#         y_true = np.argmax(t, axis=1)
#         data_size = x.shape[0]

#         correct_count = np.sum([y_true == y_pred]) 
#         score = correct_count / data_size * 100

#         return round(score, 2)
    
#     def loss(self, x, t):
#         y = self.predict(x)
        
#         return self.lastLayer.forward(y, t)
    
    
#     def optimize(self, x, t):
        
#         # forward 
#         self.loss(x, t)
        
#         # backward
#         dout = self.lastLayer.backward(1)
#         layers = list(self.layers.values())
#         layers.reverse()
#         for layer in layers:
#             dout = layer.backward(dout)
            
#         # optimizeメソッドがある層は更新を行う
#         # AffineとBatchNorm層のみ行うはず
#         for layer in self.layers.values():
#             if hasattr(layer, "optimize"):
#                 layer.optimize()
            

### TODO : Gradient Checkを導入する

In [14]:
# from sklearn.model_selection import train_test_split


# class DNN:
#     def __init__(self, init='gauss', iteration = 500, lr = 0.05,  lam = 0.01, 
#                  batch_mode = 'mini', activation='relu',
#                  batch_size_rate = 0.1, hidden_layer_list = [5], optimizer='sgd',
#                  batch_norm=False, dropout_ratio=0.0):
#         """ ハイパーパラメータ解説
#         init: 初期化方法
#             'he' : 
#             'gauss' 
#             'xavier'
#         lr : 学習率
#         lam : 正則化項の率
#         batch_size: バッチサイズ
#             'batch' : フルサイズ
#             'mini' 0< x< 1: フルサイズ割合 0.1なら全体の0.1サイズ使用する
#             'online' : オンライン学習　１データのみ
#         hidden_layer_list : 隠れ層のリスト、層のユニットをリストで入力　例[2, 3]　ユニット数２、ユニット数３の隠れ層
#         optimizer : 勾配の更新手法
#             'sgd' : 確率的勾配降下法
#             'adam': 
#             'adagrad':
#         activation: 活性化関数の名前
#             'relu' : ReLU関数
#             'tanh' : tanh
#             'sigmoid' : シグモイド関数
#         """
#         self.params = {}
#         self.params['iteration'] = iteration
#         self.params['init'] = init
#         self.params['lr'] = lr
#         self.params['lam'] = lam # 正則化項用の係数　今は使っていない
#         self.params['batch_mode'] = batch_mode # データ数が決まったらそれに基づいて変更する
#         self.params['batch_size_rate'] = batch_size_rate # ミニバッチ法のときのみ使用する
#         self.params['hidden_layer_list'] = hidden_layer_list
#         self.params['optimizer'] = optimizer
#         self.params['batch_norm'] = batch_norm
#         self.params['dropout_ratio'] = dropout_ratio
#         self.params['activation'] = activation # 活性化関数
        
#     def train(self, X, y, params={}):
#         # 入力パラメータがあれば更新する
#         for key in params:
#                 self.params[key] = params[key]
        
#         # 正規化　必要？
#         X = X / 255.0
        
#         # 訓練とテストデータに分割
#         X_train, X_test, y_train, y_test = \
#             train_test_split(X, y, test_size=0.2, random_state=0)

#         self.params['data_size'] = X_train.shape[0]
#         self.params['input_size'] = X_train.shape[1]
#         self.params['output_size'] = y_train.shape[1]

        
#         # コストや正答率の学習曲線を引くためのリストを用意
#         past_train_costs = []
#         past_test_costs = []
#         past_train_accuracy = []
#         past_test_accuracy = []
        
#         # 初期化
#         # 重み初期化
#         # バッチサイズの設定
#         if self.params['batch_mode'] == 'batch':
#             self.params['batch_size'] = self.params['data_size']
#         elif self.params['batch_mode'] == 'mini':
#             self.params['batch_size'] = int(self.params['data_size']  * self.params['batch_size_rate'] ) 
#         else:
#             self.params['batch_size'] = 1
#         # 隠れ層やレイヤーインスタンス生成
#         self.params['layer'] = Layers(self.params)
        
        
#         # 何イテレーションで1エポックか
#         epoch_per_i = int(self.params['data_size'] / self.params['batch_size'])
        
#         ##################
#         # 最急降下法での学習
#         ##################
#         for i in range(self.params['iteration']):
            
#             # 学習に使用するデータをサンプリング
#             choice_index = np.random.choice(self.params['data_size'], self.params['batch_size'])
#             X_batch, y_batch = X_train[choice_index], y_train[choice_index]
            
#             # 誤差逆伝播法によって勾配を求め、値を更新
#             self.params['layer'].optimize(X_batch, y_batch)
            
#             # 1エポックごとに正答率とコストを算出して保存する
#             if i % epoch_per_i == 0:              
#                 past_train_accuracy.append(self.params['layer'].accuracy(X_train, y_train))
#                 past_test_accuracy.append(self.params['layer'].accuracy(X_test, y_test))
                
#                 past_train_costs.append(self.params['layer'].loss(X_train, y_train))
#                 past_test_costs.append(self.params['layer'].loss(X_test, y_test))
            
#         return past_train_accuracy, past_test_accuracy, past_train_costs, past_test_costs 
    

        
#     # 現在のパラメータで予測値を確率かラベルで出力する。
#     def predict(self, X, probability=False):
#         predict = self.params['layer'].predict(X, train_flg=False)
#         predict_proba = softmax(predict)
#         if probability== True:
#             return predict_proba
#         else:
#             return np.argmax(predict_proba, axis=1)
        
#     def plot_learning_curve(self, X, y, metrics='acc', params={}): 
#         past_train_accuracy, past_test_accuracy, past_train_costs, past_test_costs = self.train(X, y, params)
#         plt.figure(figsize=(6,4))
#         # count_epoch = self.params['iteration'] // self.params['data_size'] + 1
#         if metrics == 'cost':
#             plt.plot(past_train_costs, color='orange', label='train')
#             plt.plot(past_test_costs, color='lime', label='test')
#             plt.ylabel("cost", fontsize=15)
#             print("last train cost is {}".format(past_train_costs[-1]))
#             print("last test cost is {}".format(past_test_costs[-1]))
#         else:
#             #plt.plot(np.array(past_train_accuracy), color='r')
#             plt.plot(past_train_accuracy, color='orange', label='train')
#             plt.plot(past_test_accuracy, color='lime', label='test')
#             plt.ylabel("accuracy", fontsize=15)
#             print("last train accuracy is {}".format(past_train_accuracy[-1]))
#             print("last test accuracy is {}".format(past_test_accuracy[-1]))
#             plt.ylim(-0.5, 100.5)

#         plt.legend()
#         plt.title('Learning Curve', fontsize=20)
#         plt.xlabel("iteration[epoch]", fontsize=15)
        

### プロットはクラスに入れず、trainの返り値を使ってplotすればよい。

In [15]:
# model = DNN(iteration=10, optimizer='adam', 
#             hidden_layer_list = [100], batch_norm=True)

In [16]:
# params = {'optimizer': 'adam',
#          'batch_mode' : 'mini',
#          'init': 'he',
#          'lr': 0.007}

# #past_train_accuracy, past_test_accuracy = model.train(X,y, params)
# # model.plot_learning_curve(X,y, 'cost', params)

# model.plot_learning_curve(x_train,y_train, params)

### 提出用ファイル作成

In [17]:
# コメントアウトを外せば提出ファイルができる。
# test_df = pd.read_csv("test.csv")

# X_sub = test_df
# X_sub = np.array(X_sub)

# # train_flgでエラー中　すべてparamsに押し込めれば良い
# Y_pred = model.predict(X_sub)

# submission = pd.DataFrame({
#        "ImageId": np.array(test_df.index) + 1,
#        "Label": Y_pred
#    })

# submission.to_csv("./submission_001.csv", index=False)

### 入出力サイズ対応が必要か

### DNNクラス

### Layersクラス

### 単層クラス

- Affine層
    - SGD
    - Adam
    - adgrad
- 活性化関数
    - tanh
    - ReLU
    - sigmoid
- バッチノーマリゼーション層
- ドロップアウト層(要修正)


- 畳み込み層　（forward確認中）
- プーリング層 （forward、　）
- Flatten層 (済)

im2col col2im



In [18]:
params = {}
params['activation'] = 'relu'
act1 = Activation(params)


In [19]:
print(x_train.shape)
print(act1.forward(x_train).shape)

(80, 28, 28)
(80, 28, 28)


In [20]:
print(x_train_cifar10.shape)
print(act1.forward(x_train_cifar10).shape)

(100, 32, 32, 3)
(100, 32, 32, 3)


### 活性化関数はサイズを気にしない

In [21]:
x_train_cifar10[0].shape

(32, 32, 3)

### Flattenクラス

In [205]:
class Flatten(Layer):
    # (N, H, W, C)のデータを
    # (N, H * W * C)に変更する
    def __init__(self, params={}):
        super(Flatten, self).__init__(params)
        # 入出力サイズ以外のパラメータはない
        
    def initialize(self, in_shape, params={}):
        self.in_shape = in_shape # N H W Cで来ると想定
        N, H, W, C = in_shape
        self.out_shape = (N, H * W *C) 
            
        # 次の層の初期化のために出力shapeを返す
        print("Flatten : out {} ".format(self.out_shape))
        return self.out_shape
    
    def forward(self, x):
        self.input_shape = x.shape
        out = np.array([elem.flatten() for elem in x])
        return out
    
    def backward(self, dout):
        dout = dout.reshape(self.input_shape)
        
        return dout

In [23]:
x_train[:, :, :, np.newaxis].shape

(80, 28, 28, 1)

In [24]:
flat = Flatten()
forward_val = flat.forward(x_train[:, :, :, np.newaxis])
forward_val.shape

(80, 784)

In [25]:
flat.backward(forward_val).shape

(80, 28, 28, 1)

### 出力が変わらないパディング

フィルターサイズが奇数であることを条件に、
フィルターサイズを２で割った商をパディング数とすればよい。

In [26]:
filter_size = 3

pad = filter_size // 2

print(x_train_cifar10[0].shape)
      
print(np.pad(x_train_cifar10,[(0,0),(pad, pad),(pad, pad),(0,0)],'constant').shape)

(32, 32, 3)
(100, 34, 34, 3)


### 同じように'edge'にすればパディングは隣接値をそのまま使う

In [27]:
print(np.pad(x_train_cifar10,[(0,0),(pad, pad),(pad, pad),(0,0)],'edge').shape)

(100, 34, 34, 3)


### im2colの作成

### これをim2colの反対のcol2imで元のデータ形状に戻してやれば良い

In [28]:
def im2col(x, filter_size=4, stride=4, pad=0, padding='constant'):
    # 4次元データを2次元データに変形する
    # XXX: もしかしたらできているかも？？　今の所パディングして数が変わるとエラーになる
    # TODO: 上記を修正して、畳み込み層にも使えるように修正する
    
    # パディングして外堀を埋める
    if pad > 0: 
        x = np.pad(x,[(0,0),(pad, pad),(pad, pad),(0,0)], padding)

    # データ数　高さ　幅　チャネルを取得
    N, H, W, C = x.shape
    
    # それぞれのブロックごとにflatten
    return np.array([x[n, h:h+filter_size, w:w+filter_size, c].flatten() for c in range(C) for n in range(N) \
            for h in range(0, H-filter_size+1, stride) for w in range(0, W-filter_size+1, stride)])

In [110]:
# im2colの逆をやる関数
def col2im(x, input_shape, filter_size=4, stride=4, pad=0, padding='constant'):
    
    # 戻すデータの形状
    N, H, W, C = input_shape
    
    # リターン箱を作る
    img = np.zeros(input_shape)
    # ブロック数
    block_num = (pad * 2 + W - filter_size)// stride + 1 # 本当は高さ　幅別々に計算する

    # １チャネルに何行あるか
    c_vol = int(x.shape[0] / C)
    # c_vol = int(ret.shape[0] / C)
    n_vol = int(c_vol / N)

    for i, line in enumerate(x):
        # 帯からブロックにする
        block = line.reshape(filter_size, filter_size)

        channel_i = i//c_vol

        data_i = i % c_vol // n_vol

        start_h, start_w = divmod(i % n_vol, block_num) # 本当はblock_h
        start_h, start_w = int(start_h), int(start_w)

        end_h = start_h +  filter_size# filter_size or filter_height
        end_w = start_w + filter_size # filter_size or filter_weight
#         print(block.shape)
#         print(img[data_i, start_h:end_h, start_w:end_w, channel_i].shape)
        #print("{} {} {} {} ".format(data_i, start_h, start_w, channel_i))
        img[data_i, start_h:end_h, start_w:end_w, channel_i] = block
        
    return img[:, pad:pad+H, pad:pad+W, :]                

In [30]:
ret = im2col(x_train_cifar10[:10], 3, 1, 0)
x_train_cifar10[:10].shape

(10, 32, 32, 3)

In [31]:
ret.shape

(27000, 9)

In [32]:
col2im(ret, x_train_cifar10[:10].shape, 3, 1, 0).shape

(10, 32, 32, 3)

### col2im im2colが完成！

### MaxPoolingクラス

In [204]:
class MaxPooling(Layer):
    
    def __init__(self, pool_size=4, stride=-1, pad=0, params={}):
        super(MaxPooling, self).__init__(params)
        self.pool_size = pool_size
        self.stride = stride
        self.pad = pad
        self.stride = self.pool_size if self.stride == -1 else stride
        # パディング方法をしてできたほうがよい？　今はゼロ埋め固定
        # 最大値の場所を保存して、backwardでその場所以外は伝播しない
        self.max_index = None
        self.input_shape = None # 親クラスで定義するのでここには後々必要なくなる
        
    def initialize(self, in_shape, params={}):
        self.in_shape = in_shape # N H W Cで来ると想定
        N, H, W, C = in_shape
        # 出力の高さと幅を計算する
        out_h = 1 + int((H + 2*self.pad - self.pool_size) / self.stride)
        out_w = 1 + int((W + 2*self.pad - self.pool_size) / self.stride)
        
        self.out_shape = (N, out_h, out_w, C)
            
        # 次の層の初期化のために出力shapeを返す
        print("Pooling : out {} filter {} stride {} pad {} ".format(self.out_shape, self.pool_size, self.stride, self.pad))
        return self.out_shape
        
    def forward(self, x):
        
        self.input_shape = x.shape
        N, H, W, C = x.shape
        
        col = im2col(x, self.pool_size, self.stride, self.pad)
        col_max = np.max(col, axis=1)
        self.max_index = np.argmax(col, axis=1)
        
        # 出力サイズを確認
        out_h = (H + 2*self.pad - self.pool_size)// self.stride + 1 
        out_w = (W + 2*self.pad - self.pool_size)// self.stride + 1 
        
        # 整形
        return col_max.reshape(C, N, out_h, out_w).transpose(1, 2, 3, 0)
    
    
    def backward(self, dout):

        filter_size = dout.shape[1]
        dout_line = dout.transpose(3, 0, 1, 2).reshape(-1)
        
        # 返り値の箱を作る
        ret = np.zeros([dout_line.shape[0], self.pool_size* self.pool_size])
        
        # 最大値の場所にdoutを流し込む
        for i, max_i in enumerate(poollayer.max_index):
            ret[i, max_i] = dout_line[i]
            
        # 元の形状に戻してリターン
        return col2im(ret, self.input_shape, self.pool_size, self.stride, self.pad)

In [265]:
class Convolution(Layer):
    
    def __init__(self, out_channel = 1, filter_size=3, stride=1, pad=0, bias=True, params={}):
        # biasなしに対応していない。 dbを更新しなければよい？
        self.out_channel =out_channel
        self.filter_size = filter_size
        self.stride = stride
        self.pad = pad
        self.bias = bias
        self.W = None
        self.b = None
        ##### Affineからパクリ ########
        self.x = None
        # パラメータの微分値
        self.dW = None
        self.db = None
        self.optimize = None
        # 学習率のセット
        if 'lr' in params:
            self.lr = params['lr']
        else:
            self.lr = 0.01
        self.x_2dim = None # im2col後のxの値を保持
        
        
    def initialize(self, in_shape, params={}):
        self.in_shape = in_shape # N H W Cで来ると想定
        N, H, W, C = in_shape
        # 出力の高さと幅を計算する
        out_h = 1 + int((H + 2*self.pad - self.filter_size) / self.stride)
        out_w = 1 + int((W + 2*self.pad - self.filter_size) / self.stride)
        
        self.out_shape = (N, out_h, out_w, self.out_channel)
        
        # 重みの初期化
        # im2colを見越して２次元の重みとして実装
        self.W = np.random.randn(self.filter_size * self.filter_size * self.in_shape[3], self.out_channel)
        self.b = np.zeros([self.out_channel, 1])
        # TODO 初期化を選択できるように設定する　ひとまずはガウス初期化
#         if params['init'] == 'gauss':
        self.W *= 0.01
#         elif params['init'] == 'xavier':
#             # 入力層のユニット数 N * out_h * out_w ? 
#             self.W /= np.sqrt(self.out_shape[0] * self.out_shape[1] * self.out_shape[2])
#         else: # He
#             self.W = self.W / np.sqrt(self.out_shape[0] * self.out_shape[1] * self.out_shape[2]) * np.sqrt(2) 
        
        # 更新式のスイッチング
        # optimizeメソッドをoptimizerによって切り替える。
        if 'optimizer' in params:
            if params['optimizer']=='sgd':
                self.optimize = self.update_sgd
            elif params['optimizer'] == 'adagrad':
                self.h = np.zeros_like(W)
                self.optimize = self.update_adagrad
            else: # params['optimizer'] == 'adam':
                self.m = np.zeros_like(W)
                self.v = np.zeros_like(W)
                self.beta1 = 0.9
                self.beta2 = 0.999
                self.optimize = self.update_adam
        else:
            self.optimize = self.update_sgd
            
        # 次の層の初期化のために出力shapeを返す
        print("Conv : out {} filter {} stride {} pad {} ".format(self.out_shape, self.filter_size, self.stride, self.pad))
        return self.out_shape
        
    def forward(self, x):
        in_C = self.in_shape[3]
        out_C = self.out_shape[3]
        N = self.in_shape[0]
        
        # 出力の高さと幅
        out_h, out_w = self.out_shape[1], self.out_shape[2]

        # ２次元配列に変換する
        # (out_h * out_w * N * C, filter_size*filter_size)
        print("x.shape {}".format(x.shape))
        x_2dim = im2col(x, self.filter_size, self.stride, self.pad)
        print("x_2dim.shape {}".format(x_2dim.shape))
        print("filter_size {} stride {} pad {}".format(self.filter_size, self.stride, self.pad))
        self.x_2dim = x_2dim
        
        # 各色（チャネル）ごとに重みを分割
        W_color = self.W.reshape(in_C, -1, out_C)
        print("in_C {} out_h {} out_w {} N {} ".format(in_C, out_h, out_w, N))
        x_2dim_color = x_2dim.reshape(in_C, -1, out_h * out_w * N)
        
        # 入力チャネルごとに対応するフィルターで計算する
        out = np.zeros((in_C, out_C, out_h * out_w * N))
        for c in range(in_C):
            out[c] = np.dot(W_color[c].T, x_2dim_color[c])
        
        # 入力チャネルごとの結果を合計する
        out = np.sum(out, axis=0)
        
        out = out + self.b
        
        # 整形する　out_C N H W
        return  out.reshape(out_C, N, out_h, out_w).transpose(1, 2 ,3, 0)
        
    
    def backward(self, dout):
        out_c = self.out_shape[3]
        dout = dout.reshape(-1, out_c)
        
        self.db = np.sum(dout, axis=0)[:, np.newaxis]

        # 入力チャネルの数
        in_C = self.in_shape[3]
        # sum前の状態に戻す　（入力チャネル分複製する）
        dout_before_sum = np.tile(dout, (in_C, 1))
        self.dW = np.dot(self.x_2dim.T, dout_before_sum)

        dout = np.dot(dout, self.W.T)

        # (R G B)と横並びなので
        #  R
        #  G
        #  B　と縦方向に変換する
        dout = np.vstack(np.split(dout, in_C, axis=1))

        dout = col2im(dout, self.in_shape, self.filter_size, self.stride, self.pad)
        return dout
    
    #####affineからパクリ
    def update_sgd(self):
        self.W -= self.lr * self.dW
        self.b -= self.lr * self.db
    
    
    # adagrad 少しずつ更新量が減っていく
    def update_adagrad(self, lr = 0.01):
        self.h += self.dW ** 2
        self.W -= self.lr * self.dW / (np.sqrt(self.h) + 1e-7)
        self.b -= self.lr * self.db
        
    
    def update_adam(self, lr = 0.01):
        self.m = self.beta1 * self.m + (1- self.beta1) * self.dW
        self.v = self.beta2 * self.v + (1- self.beta2) * (self.dW * self.dW)
        
        m_hat = self.m / (1 - self.beta1)
        v_hat = self.v / (1 - self.beta2)
        
        self.W -= self.lr * m_hat / (np.sqrt(v_hat) + 1e-8)
        self.b -= self.lr * self.db

### convolutionクラスの確認

In [192]:
conv = Convolution(2, 3, 1, 0)
data = x_train_cifar10[:10]
print(data.shape)
conv.initialize(data.shape)
out = conv.forward(data)
print(out.shape)

(10, 32, 32, 3)
Conv : out (10, 30, 30, 2) filter 3 stride 1 pad 0 
shape(10, 30, 30, 2)
(10, 30, 30, 2)


In [193]:
dout_check = conv.backward(out)
dout_check.shape

(10, 32, 32, 3)

### Poolingクラスの確認

In [194]:
poollayer = MaxPooling(pool_size=3,stride=3, pad=0)
print(x_train_cifar10[:10].shape)
dout = poollayer.forward(x_train_cifar10[:10])
dout.shape

(10, 32, 32, 3)
shape(10, 10, 10, 3)


(10, 10, 10, 3)

In [195]:
d = poollayer.backward(dout)
d.shape

(10, 32, 32, 3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
conv2d_1 (Conv2D)            (None, 28, 28, 2)         20        
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 2)         8         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 2)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 2)         38        
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 14, 2)         8         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 2)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 2)           38        
_________________________________________________________________
batch_normalization_3 (Batch (None, 7, 7, 2)           8         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 3, 3, 2)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               2432      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290  

In [270]:
from collections import OrderedDict

class LeNetLayers:
    def __init__(self, params):
        unit_size_list = [params['input_size']]
        unit_size_list.extend(params['hidden_layer_list'])
        unit_size_list.append(params['output_size'])
        
        self.params = {}
        
        # レイヤの生成
        self.layers = OrderedDict()
        
        # とりあえずここにべた書きで書けるようにする
        self.layers['Conv1'] = Convolution(6, 5, 1, 2)# 2 (3,3) (1,1) same
        # self.layers['BatchNorm1'] = BatchNorm(params)
        self.layers['Active1'] = Activation(params) # relu
        self.layers['Pool1'] = MaxPooling(2, 2, 0, params) 
        
        self.layers['Conv2'] = Convolution(16, 5, 1, 2)# 2 (3,3) (1,1) same
        # self.layers['BatchNorm2'] = BatchNorm(params)
        self.layers['Active2'] = Activation(params) # relu
        self.layers['Pool2'] = MaxPooling(2, 2, 0, params) 
        
        self.layers['Flatten'] = Flatten(params) 
        
        # アフィン変換層（Wx + b）を追加する
        self.layers['Affine1'] = Affine(120, params)
        self.layers['Active3'] = Activation(params) # relu
        self.layers['Affine2'] = Affine(84, params)
        self.layers['Active4'] = Activation(params) # relu
        self.layers['Affine3'] = Affine(params['output_size'], params)

        
    def initialize(self, x, y, params):
        in_shape = x.shape # N, H, W, C
        print(x.shape)
        out_shape = y.shape # N, Class
        print('###########' * 3)
        for i, layer in enumerate(self.layers.values()):
            print(" Layer {}".format(i))
            in_shape = layer.initialize(in_shape, params)
            print('###########' * 3)
        
        
#         for i in range(1, len(unit_size_list)):
#             # 重みの初期化
#             init_W = np.random.randn(unit_size_list[i-1], unit_size_list[i])
#             init_b = np.zeros([1, unit_size_list[i]])
#             if params['init'] == 'gauss':
#                 init_W *= 0.01
#             elif params['init'] == 'xavier':
#                 init_W /= np.sqrt(unit_size_list[i-1])
#             else: # He
#                 init_W = init_W / np.sqrt(unit_size_list[i-1]) * np.sqrt(2) 
                
#             # アフィン変換層（Wx + b）を追加する
#             self.layers['Affine' + str(i)] = Affine(init_W, init_b, params)
            
#             # 最終層以外はバッチノーマリゼーション層と活性化関数層を追加する
#             if i < (len(unit_size_list)-1):
#                 if params['batch_norm'] == True:
#                     self.layers['BatchNorm' + str(i)] = BatchNorm(params)
#                 self.layers['Active' + str(i)] = Activation(params)
# #                 if params['dropout_ratio'] > 0:
# #                     self.layers['Dropout' + str(i)] = Dropout(params)
        
        self.lastLayer = SoftmaxWithLoss()
        
        # self.params['hidden_layer_num'] = len(unit_size_list)-1
        
    def predict(self, x):
        # forwardを繰り返す
        # ソフトマックスを通さなくても答えは出るのでこれで予測とする 
        # argmaxでラベルを取れる
        for layer in self.layers.values():
            x =layer.forward(x)

        return x

    def accuracy(self, x, t):
        # 正答率を小数点第二桁で出力する
        y_pred = self.predict(x)
        y_pred = np.argmax(y_pred, axis=1)
        y_true = np.argmax(t, axis=1)
        data_size = x.shape[0]

        correct_count = np.sum([y_true == y_pred]) 
        score = correct_count / data_size * 100

        return round(score, 2)
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y, t)
    
    
    def optimize(self, x, t):
        
        # forward 
        self.loss(x, t)
        
        # backward
        dout = self.lastLayer.backward(1)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # optimizeメソッドがある層は更新を行う
        # AffineとBatchNorm層のみ行うはず
        for layer in self.layers.values():
            if hasattr(layer, "optimize"):
                layer.optimize()
            

In [271]:
from sklearn.model_selection import train_test_split


class DNN:
    def __init__(self, init='gauss', iteration = 500, lr = 0.05,  lam = 0.01, 
                 batch_mode = 'mini', activation='relu',
                 batch_size_rate = 0.1, hidden_layer_list = [5], optimizer='sgd',
                 batch_norm=False, dropout_ratio=0.0):
        """ ハイパーパラメータ解説
        init: 初期化方法
            'he' : 
            'gauss' 
            'xavier'
        lr : 学習率
        lam : 正則化項の率
        batch_size: バッチサイズ
            'batch' : フルサイズ
            'mini' 0< x< 1: フルサイズ割合 0.1なら全体の0.1サイズ使用する
            'online' : オンライン学習　１データのみ
        hidden_layer_list : 隠れ層のリスト、層のユニットをリストで入力　例[2, 3]　ユニット数２、ユニット数３の隠れ層
        optimizer : 勾配の更新手法
            'sgd' : 確率的勾配降下法
            'adam': 
            'adagrad':
        activation: 活性化関数の名前
            'relu' : ReLU関数
            'tanh' : tanh
            'sigmoid' : シグモイド関数
        """
        self.params = {}
        self.params['iteration'] = iteration
        self.params['init'] = init
        self.params['lr'] = lr
        self.params['lam'] = lam # 正則化項用の係数　今は使っていない
        self.params['batch_mode'] = batch_mode # データ数が決まったらそれに基づいて変更する
        self.params['batch_size_rate'] = batch_size_rate # ミニバッチ法のときのみ使用する
        self.params['hidden_layer_list'] = hidden_layer_list
        self.params['optimizer'] = optimizer
        self.params['batch_norm'] = batch_norm
        self.params['dropout_ratio'] = dropout_ratio
        self.params['activation'] = activation # 活性化関数
        
    def train(self, X, y, params={}):
        # 入力パラメータがあれば更新する
        for key in params:
                self.params[key] = params[key]
        
        # 正規化　必要？
        X = X / 255.0
        
        # 訓練とテストデータに分割
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=0.2, random_state=0)

        self.params['data_size'] = X_train.shape[0]
        self.params['input_size'] = X_train.shape[1]
        self.params['output_size'] = y_train.shape[1]

        
        # コストや正答率の学習曲線を引くためのリストを用意
        past_train_costs = []
        past_test_costs = []
        past_train_accuracy = []
        past_test_accuracy = []
        

        # バッチサイズの設定
        if self.params['batch_mode'] == 'batch':
            self.params['batch_size'] = self.params['data_size']
        elif self.params['batch_mode'] == 'mini':
            self.params['batch_size'] = int(self.params['data_size']  * self.params['batch_size_rate'] ) 
        else:
            self.params['batch_size'] = 1
        # 隠れ層やレイヤーインスタンス生成
        self.params['layer'] = LeNetLayers(self.params)
        # 入出力サイズ
        print("before layer making {}".format(X_train.shape))
        self.params['layer'].initialize(X_train, y_train, self.params)
        
        # print(self.params['layer'].predict(X_train).shape)
        # 確認のためここでブレークする
#         if True:
#             return self.params['layer'].predict(X_train)
        
        # 何イテレーションで1エポックか
        epoch_per_i = int(self.params['data_size'] / self.params['batch_size'])
        
        ##################
        # 最急降下法での学習
        ##################
        for i in range(self.params['iteration']):
            
            # 学習に使用するデータをサンプリング
            choice_index = np.random.choice(self.params['data_size'], self.params['batch_size'])
            X_batch, y_batch = X_train[choice_index], y_train[choice_index]
            
            # 誤差逆伝播法によって勾配を求め、値を更新
            self.params['layer'].optimize(X_batch, y_batch)
            
            # 1エポックごとに正答率とコストを算出して保存する
            if i % epoch_per_i == 0:              
                past_train_accuracy.append(self.params['layer'].accuracy(X_train, y_train))
                past_test_accuracy.append(self.params['layer'].accuracy(X_test, y_test))
                
                past_train_costs.append(self.params['layer'].loss(X_train, y_train))
                past_test_costs.append(self.params['layer'].loss(X_test, y_test))
            
        return past_train_accuracy, past_test_accuracy, past_train_costs, past_test_costs 
    

        
    # 現在のパラメータで予測値を確率かラベルで出力する。
    def predict(self, X, probability=False):
        predict = self.params['layer'].predict(X, train_flg=False)
        predict_proba = softmax(predict)
        if probability== True:
            return predict_proba
        else:
            return np.argmax(predict_proba, axis=1)
        
    def plot_learning_curve(self, X, y, metrics='acc', params={}): 
        past_train_accuracy, past_test_accuracy, past_train_costs, past_test_costs = self.train(X, y, params)
        plt.figure(figsize=(6,4))
        # count_epoch = self.params['iteration'] // self.params['data_size'] + 1
        if metrics == 'cost':
            plt.plot(past_train_costs, color='orange', label='train')
            plt.plot(past_test_costs, color='lime', label='test')
            plt.ylabel("cost", fontsize=15)
            print("last train cost is {}".format(past_train_costs[-1]))
            print("last test cost is {}".format(past_test_costs[-1]))
        else:
            #plt.plot(np.array(past_train_accuracy), color='r')
            plt.plot(past_train_accuracy, color='orange', label='train')
            plt.plot(past_test_accuracy, color='lime', label='test')
            plt.ylabel("accuracy", fontsize=15)
            print("last train accuracy is {}".format(past_train_accuracy[-1]))
            print("last test accuracy is {}".format(past_test_accuracy[-1]))
            plt.ylim(-0.5, 100.5)

        plt.legend()
        plt.title('Learning Curve', fontsize=20)
        plt.xlabel("iteration[epoch]", fontsize=15)
        

In [272]:
 model = DNN(iteration=10, optimizer='adam', 
         hidden_layer_list = [100], batch_norm=True)

params = {'optimizer': 'adam',
#          'batch_mode' : 'mini',
#          'init': 'he',
         'lr': 0.01}


In [273]:
# x_digit = x_train[:, :, :, np.newaxis]
# y_digit = y_train

In [274]:
# y_train_cifar10.reshape(100,10)

In [275]:
pred_y = model.train(x_train_cifar10, y_train_cifar10.reshape(100,10), params)

before layer making (80, 32, 32, 3)
(80, 32, 32, 3)
#################################
 Layer 0
Conv : out (80, 32, 32, 6) filter 5 stride 1 pad 2 
#################################
 Layer 1
Activation : out (80, 32, 32, 6)   func : relu 
#################################
 Layer 2
Pooling : out (80, 16, 16, 6) filter 2 stride 2 pad 0 
#################################
 Layer 3
Conv : out (80, 16, 16, 16) filter 5 stride 1 pad 2 
#################################
 Layer 4
Activation : out (80, 16, 16, 16)   func : relu 
#################################
 Layer 5
Pooling : out (80, 8, 8, 16) filter 2 stride 2 pad 0 
#################################
 Layer 6
Flatten : out (80, 1024) 
#################################
 Layer 7
Affine : out (80, 120) optimizer adam unit 120 
#################################
 Layer 8
Activation : out (80, 120)   func : relu 
#################################
 Layer 9
Affine : out (80, 84) optimizer adam unit 84 
#################################
 Layer 10
A

ValueError: cannot reshape array of size 614400 into shape (3,newaxis,81920)